# Get all classes 

In [1]:
import glob

files = glob.glob("*.kicad_sch")


find = '(property "Netclass"'
arr=[]

for name in files: 
    with open(name) as f: 
        txt= f.read()

        for l in txt.split("\n"): 
            if find in l: 
                #print(l.replace(find, ""))
                arr.append(l.replace(find, ""))

net_classes = [x.strip().replace('"',"") for x in set(arr)]
net_classes

['hvn', '48v', 'hvp', 'gnd', 'hvm']

# Make insulation table 

In [3]:
import glob

files = glob.glob("*.kicad_sch")


find = '(property "Netclass"'
arr=[]

for name in files: 
    with open(name) as f: 
        txt= f.read()

        for l in txt.split("\n"): 
            if find in l: 
                #print(l.replace(find, ""))
                arr.append(l.replace(find, ""))

net_classes = [x.strip().replace('"',"") for x in set(arr)]
net_classes

['hvn', '48v', 'hvp', 'gnd', 'hvm']

In [4]:
import pandas as pd
df = pd.DataFrame(index = net_classes, columns = net_classes)
df.to_csv("insulation_dist.csv")
df

,hvn,48v,hvp,gnd,hvm
hvn,NaN,NaN,NaN,NaN,NaN
48v,NaN,NaN,NaN,NaN,NaN
hvp,NaN,NaN,NaN,NaN,NaN
gnd,NaN,NaN,NaN,NaN,NaN
hvm,NaN,NaN,NaN,NaN,NaN


In [6]:
# manual modification 
df = pd.read_csv("insulation_dist_v0.csv", index_col=0)
df

,gnd,48v,hvn,hvm,hvp
gnd,0.5,NaN,NaN,NaN,NaN
48v,1.0,1.0,NaN,NaN,NaN
hvn,4.0,4.0,0.5,NaN,NaN
hvm,4.0,4.0,3.0,2.0,NaN
hvp,4.0,4.0,3.0,3.0,0.5


### CustomRules

In [7]:
txt1= """
# Comment__
(rule "RuleName__"
  (constraint clearance (min dist__mm))
  (condition "A.NetClass == 'net__' && B.NetClass == 'net__' && A.NetName != B.NetName")
)
"""

txt2= """
# Comment__
(rule "RuleName__"
  (constraint clearance (min dist__mm))
  (condition "(A.NetClass == 'net1__' && B.NetClass == 'net2__') || (A.NetClass == 'net2__' && B.NetClass == 'net1__')")
)
"""

In [8]:
import uuid

# Generate a random UUID
uid = str(uuid.uuid4())
uid

'28e74442-c10f-411d-a73a-e22dcbbacc84'

In [9]:
txt = "(version 1)\n\n"
index = list(df.index)
for i in range(len(index)):
    for j in range(0,i+1):
        p1= index[i]
        p2= index[j]
        d= df.iloc[i, j]

        uid = str(uuid.uuid4())

        if p1== p2: 
            print("=", p1,p2,d)
            txt+=txt1.replace("net__", p1).\
                replace("dist__", str(d)).\
                replace("Comment__", "Self Insulation" +p1 ).\
                replace("RuleName__", uid)
        else : 
            print("#" , p1,p2,d)
            txt+=txt2.replace("net1__", p1).\
                replace("net2__", p2).\
                replace("dist__", str(d)).\
                replace(f"Comment__", f"Class to class insulation {p1} vs {p2}" ) .\
                replace("RuleName__", uid)
with open("CustomRules.txt", "w") as f:
    f.write(txt)

= gnd gnd 0.5
# 48v gnd 1.0
= 48v 48v 1.0
# hvn gnd 4.0
# hvn 48v 4.0
= hvn hvn 0.5
# hvm gnd 4.0
# hvm 48v 4.0
# hvm hvn 3.0
= hvm hvm 2.0
# hvp gnd 4.0
# hvp 48v 4.0
# hvp hvn 3.0
# hvp hvm 3.0
= hvp hvp 0.5


In [10]:
# syntaxe